In [1]:
import numpy as np
import pandas as pd
import scipy
import statsmodels
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import plotly
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import chart_studio.plotly as py
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import docx
import datetime as dt
import pickle
import random
import math
import time
import os
import re
import colorsys
import arrow
import locale
import requests
import collections
import numbers
import decimal

from warnings import simplefilter

from scipy import stats
from scipy.interpolate import make_interp_spline, BSpline
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from matplotlib.lines import Line2D
import matplotlib.dates as mdates
from pathlib import Path

In [2]:
def saturate_color(color, saturation=0.75):

    if isinstance(color, str):
        color_rgb = mpl.colors.to_rgb(color)
    else:
        color_rgb = color
    
    color_hls = colorsys.rgb_to_hls(
        color_rgb[0], color_rgb[1], color_rgb[2])
    color_hls_saturated = (
        color_hls[0], color_hls[1], saturation*color_hls[2])
    color_rgb_saturated = colorsys.hls_to_rgb(
        color_hls_saturated[0], color_hls_saturated[1], color_hls_saturated[2])
    
    return color_rgb_saturated


def saturate_palette(palette, saturation=0.75):
    palette_saturated = [saturate_color(i, saturation=saturation) for i in palette]
    return palette_saturated


def alpha_color(color, alpha=0.75):
    
    if isinstance(color, str):
        color = mpl.colors.to_rgb(color)
        
    new_color = tuple (x + (1 - x) * (1 - alpha) for x in color)
    return new_color


def alpha_palette(palette, alpha=0.75):
    palette_alphed = [alpha_color(i, alpha=alpha) for i in palette]
    return palette_alphed

In [3]:
theme = 'light'
# theme = 'dark'

scatter = {
    'lw': 0.15
}
point = {
    'linestyle': 'none',
    'markersize': 3,
    'err_kws': {'lw': 1.2},
    'capsize': 0.03
}
hist = {
    'alpha': 1
}
title_inline = {
    'pad': 27
}
custom_params = {
    'figure.figsize': (9, 2.5), # ширина и высота в дюймах
    'figure.titlesize': 12, # заголовок фигуры
    'figure.titleweight': 'medium', # вес заголовка фигуры
    'figure.labelsize': 9, # подписи фигуры 
    'figure.labelweight': 'medium',
    'axes.axisbelow': True,
    'axes.spines.bottom': True, # ось X
    'axes.spines.left': True, # ось Y
    'axes.spines.right': False, # не отображать правую рамку 
    'axes.spines.top': False, # не отображать верхнюю рамку
    'axes.linewidth': 0.5, # толщина линий осей 
    'axes.titlelocation': 'left', # расположение заголовка
    'axes.titlepad': 15.0, # расстояние между заголовком и графиком
    'axes.titlesize': 10.0, # заголовок
    'axes.titleweight': 'medium', # заголовок
    'axes.labelpad': 8.0, # расстояние между подписью оси и осью
    'axes.labelsize': 9.0, # оси
    'axes.labelweight': 'bold', # подписи осей
    'xaxis.labellocation': 'left', # положение подписи
    'yaxis.labellocation': 'top', # положение подписи
    'axes.grid': True,
    'axes.grid.axis': 'both', # какую разметку отображать (x - ветикальные линии, y - горизонтальные линии)
    'grid.linestyle': ':',
    'grid.linewidth': 0.5,
    'xtick.bottom': False,
    'xtick.direction': 'in',
    'xtick.major.bottom': True,
    'xtick.major.pad': 3.5,
    'xtick.major.size': 2.0,
    'xtick.major.top': True,
    'xtick.major.width': 1,
    'xtick.labelsize': 8.0, # деления оси X
    'ytick.left': False,
    'ytick.direction': 'in',
    'ytick.major.left': True,
    'ytick.major.pad': 3.5,
    'ytick.major.size': 2.0,
    'ytick.major.left': True,
    'ytick.major.width': 1,
    'ytick.labelsize': 8.0, # деления оси Y
    'lines.linewidth': 1.2,
    'lines.markersize': 3,
    'legend.title_fontsize': 8.0,
    'legend.fontsize': 9.0,
    'legend.framealpha': 0.75,
    'font.family': 'Ubuntu'
}
custom_params_light = {
    'figure.facecolor': '#FEFEFE', # FEFEFE
    'figure.edgecolor': '#FEFEFE',
    'axes.facecolor': '#FEFEFE',
    'axes.titlecolor': '#505050',
    'axes.labelcolor': '#505050',
    'axes.edgecolor': '#CCCCCC', # цвет осей
    'grid.color': '#D9D9D9',  
    'xtick.color': '#BBBBBB', # 0.90, BBBBBB
    'xtick.labelcolor': '#9F9F9F',
    'ytick.color': '#BBBBBB', # 0.90
    'ytick.labelcolor': '#9F9F9F',
    'legend.labelcolor': '#4B4B4B',
    'legend.edgecolor': '#A6A6A6',
    'legend.facecolor': '#FEFEFE',
    'text.color': '#4B4B4B',
    'patch.edgecolor': '#FEFEFE', # цвет границы стобцов, 
    'patch.force_edgecolor': True,
    'scatter.edgecolors': 'face'
}
custom_params_dark = {
    'figure.facecolor': '#303030',
    'axes.facecolor': '#303030',
    'axes.edgecolor': '#555555',
    'axes.titlecolor': '#AAAAAA',
    'axes.labelcolor': '#BBBBBB',
    'axes.edgecolor': '#BBBBBB', # цвет осей
    'grid.color': '#AAAAAA', # CCCCCC
    'grid.linewidth': 0.1,
    'xtick.color': '#BBBBBB',
    'ytick.color': '#BBBBBB',
    'lines.markersize': 3.5,
    'legend.edgecolor': '#555555',
    'legend.facecolor': '#303030',
    'legend.labelcolor': '#E5E5E5',
    'legend.framealpha': 0.5,
    'text.color': '#E5E5E5',
    'patch.edgecolor': '#303030'
}

if theme == 'light':

    palette_bright = sns.color_palette([
    #    0 : 0.3    1 red      2 blue     3 green    4 yellow
        '#4B4B4B', '#AF4035', '#4682B4', '#2C8B6D', '#FFA500',
    #    5 brown    6 khaki    7 purple   8 pink     9 orange
        '#6C4839', '#9A7D4D', '#534D89', '#C95576', '#DB8727',
    #   -10 dark   -9 blue    -8 blue    -7 blue    -6 ligt
        '#304E68', '#3D6384', '#4D7DA8', '#6D9BC3', '#A7CBE8',
    #   -5 : 0.2   -4 : 0.5   -3 : 0.65  -2 : 0.75  -1 : 0.85
        '#353535', '#7F7F7F', '#A5A5A5', '#BFBFBF', '#D9D9D9'
    ])
    palette = saturate_palette(palette_bright, saturation=0.85)
    palette = alpha_palette(palette, alpha=0.90)
    # add palette to custom parameters dictionary for light theme
    custom_params_light.update({'axes.prop_cycle': plt.cycler(color=palette)})

    corr_matrix = sns.color_palette(
        'blend:#255986,#FFFFFF,#933737', as_cmap=True)
    report = sns.color_palette(
        'blend:#E6E9EC,#FFFFFF,#9E2A2B', as_cmap=True)
    gradient = sns.color_palette(
        'blend:#FFFFFF,#3D6384', as_cmap=True)
    pairplot = sns.color_palette(['#8DB4D7', '#AF4035'])

    # add custom parameters of light theme to parameters dictionary
    custom_params.update(custom_params_light)
    # update rcParams 
    plt.rcParams.update(custom_params)

if theme == 'dark':
    
    palette = sns.color_palette([
        # grey     # red      # blue     # green    # yellow
        '#DDDDDD', '#DC4B3E', '#7EA8E0', '#28BF62', '#DCDE7B',
        '#9A6484', '#5B3256', '#6C4839', '#5C8DA0', '#847637',
        '#304E68', '#3D6384', '#4D7DA8', '#6D9BC3', '#A7CBE8',
        '#505050', '#606060', '#707070', '#808080', '#AAAAAA',
    ])
    # add palette to custom parameters dictionary for dark theme
    custom_params_dark.update({'axes.prop_cycle': plt.cycler(color=palette)})
    
    corr_matrix = sns.color_palette(
        'blend:#285780,#909090,#8D3131', as_cmap=True)
    report = sns.color_palette(
        'blend:#E6E9EC,#CCCCCC,#9E2A2B', as_cmap=True)
    gradient = sns.color_palette(
        'blend:#FFFFFF,#3D6384', as_cmap=True)
    pairplot = sns.color_palette(['#8DB4D7', '#AF4035'])

    # add custom parameters of dark theme to parameters dictionary
    custom_params.update(custom_params_dark)
    # update rcParams 
    plt.rcParams.update(custom_params)

%matplotlib inline

seed = 42

In [4]:
sns.color_palette(palette[:5])

[(0.36470588235294116, 0.36470588235294116, 0.36470588235294116),
 (0.6853529411764706, 0.3523529411764706, 0.3193529411764706),
 (0.3761764705882353, 0.5561764705882352, 0.7061764705882353),
 (0.2804411764705882, 0.5654411764705882, 0.4754411764705882),
 (0.9325, 0.6625, 0.16749999999999993)]

In [5]:
sns.color_palette(palette[5:10])

[(0.4676764705882353, 0.35967647058823526, 0.3146764705882353),
 (0.6231470588235294, 0.5361470588235293, 0.3921470588235293),
 (0.4056470588235293, 0.38764705882352934, 0.5676470588235294),
 (0.7787058823529412, 0.43070588235294116, 0.5297058823529411),
 (0.8387941176470588, 0.47579411764705876, 0.3317941176470588)]

In [6]:
sns.color_palette(palette[10:15])

[(0.28423529411764703, 0.37423529411764694, 0.452235294117647),
 (0.3340882352941177, 0.44808823529411757, 0.5470882352941177),
 (0.39585294117647046, 0.5398529411764704, 0.6688529411764705),
 (0.507470588235294, 0.645470588235294, 0.7654705882352942),
 (0.7066176470588236, 0.8146176470588233, 0.9016176470588235)]

In [7]:
sns.color_palette(palette[15:])

[(0.28705882352941176, 0.28705882352941176, 0.28705882352941176),
 (0.548235294117647, 0.548235294117647, 0.548235294117647),
 (0.6823529411764706, 0.6823529411764706, 0.6823529411764706),
 (0.7741176470588236, 0.7741176470588236, 0.7741176470588236),
 (0.8658823529411764, 0.8658823529411764, 0.8658823529411764)]

In [683]:
pio.templates['primetheme'] = go.layout.Template(
    layout = {
        'hovermode': 'x unified',
        'hoverlabel': dict(
            bgcolor='rgba(255,255,255,0.85)',
            bordercolor='#BBBBBB',
            font=dict(color='#454545')),
        # 'hovermode': 'x',
        # 'hoverlabel': dict(
        #     bordercolor='#FFFFFF',
        #     font=dict(color='#FFFFFF')),
        'hidesources': True,
        'autosize': False,
        # 'width': 750,
        # 'height': 250,
        'margin': dict(
            t=0, r=0, l=50, b=50
        ),
        'font' : {
            'family': 'Ubuntu',
            'size': 11,
        },
        'title': {
            'automargin': False,
            'pad': {'t': 10},
            'font': {
                'family': 'Ubuntu',
                'color': '#404040',
                'size': 15
            },
            'x': 0.1,
            'y': 1
        },
        'colorway': list(
            (
            #    0 : 0.3    1 red      2 blue     3 green    4 yellow
                '#4B4B4B', '#AF4035', '#4682B4', '#2C8B6D', '#FFA500',
            #    5 brown    6 khaki    7 purple   8 pink     9 orange
                '#6C4839', '#9A7D4D', '#534D89', '#C95576', '#DE6535',
            #   -10 dark   -9 blue    -8 blue    -7 blue    -6 ligt
                '#304E68', '#3D6384', '#4D7DA8', '#6D9BC3', '#A7CBE8',
            #   -5 : 0.2   -4 : 0.5   -3 : 0.65  -2 : 0.75  -1 : 0.85
                '#353535', '#7F7F7F', '#A5A5A5', '#BFBFBF', '#D9D9D9')
            ),
        'dragmode': 'pan',
        'xaxis': dict(
            anchor='free',
            position=0,
            color='#707070',
            ticks='outside',
            griddash='1px',
            gridwidth=1,
            gridcolor='#EBEBEB',
            linecolor='#E1E1E1',
            linewidth=1,
            ticklen=5,
            tickcolor='#FCFCFC',
            showgrid=True,
            showline=True,
            showticklabels=True,
            showspikes=True,
            spikedash='3px',
            spikecolor='#A5A5A5',
            spikesnap="cursor",
            spikemode="across",
            spikethickness=1,
            zeroline=False,
            title={
                'font': dict(
                    color='#707070'),
                'standoff': 10},
            tickangle=0,
        ),
        'yaxis': dict(
            anchor='free',
            color='#454545',
            ticks='outside',
            griddash='1px',
            gridwidth=1,
            gridcolor='#EBEBEB',
            linecolor='#E1E1E1',
            linewidth=1,
            ticklen=5,
            tickcolor='#FCFCFC',
            position=0,
            showgrid=True,
            showline=True,
            showticklabels=True,
            zeroline=False,
            title={
                'font': dict(
                    color='#707070'),
                'standoff': 10},
            tickangle=0
        ),
        'legend': dict(
            title={
                'font': {
                    'family': 'Ubuntu',
                    'color': '#4B4B4B',
                    'size': 14,
                    'style': 'normal',
                    'weight': 'bold'
                }
            },
            font={
                'family': 'Ubuntu',
                'color': '#707070',
                'size': 12,
                'style': 'normal',
                'weight': 'normal'
            },
            orientation='h',
            yanchor="bottom",
            y=1,
            xanchor="left",
            x=0,
            entrywidth=40,
            itemsizing='trace',
            itemwidth=30,
            tracegroupgap=10,
            indentation=20),
        'paper_bgcolor': 'rgba(0,0,0,0)', # прозрачный фон
        'plot_bgcolor': 'rgba(0,0,0,0)', # прозрачный фон
    },
    data = {
        # Each graph object must be in a tuple or list for each trace
        'bar': [
            go.Bar(
                texttemplate ='%{value:$.2s}',
                textposition='outside',
                textfont= {
                    'family': 'Ubuntu',
                    'size': 10,
                    'color': '#FFFFFF'
                    })
            ],
        'scatter': [
            go.Scatter(
                line={
                    'width': 1.75},
                marker={
                    'size': 3}
            )
        ]
    }
)

pio.templates.default = 'primetheme'
palette = pio.templates['primetheme'].layout.colorway

In [685]:
config = dict(
    scrollZoom=False,
    showLink=False,
    displaylogo=False,
    displayModeBar=True,
    locale='ru',
    responsive=True,
    toImageButtonOptions = {
        'format': 'png', # one of png, svg, jpeg, webp
        # 'filename': 'plot',
        'height': None,
        'width': None,
        'scale': 1 # Multiply title/legend/axis/canvas sizes by this factor
        },
    # double click on plot - reset scale
    modeBarButtonsToRemove = [
        'zoom',
        'pan',
        'select',
        'zoomIn',
        'zoomOut',
        'autoScale',
        'resetScale',
        'zoom2d',
        'pan2d',
        'select2d',
        'lasso2d',
        'zoomIn2d',
        'zoomOut2d',
        'autoScale2d',
        'resetScale2d',
        'drawline',
        'drawopenpath',
        'drawclosedpath',
        'drawcircle',
        'drawrect',
        'eraseshape',
        'zoom3d',
        'pan3d',
        'orbitRotation',
        'tableRotation',
        'handleDrag3d',
        'resetCameraDefault3d',
        'resetCameraLastSave3d',
        'hoverClosest3d',
        'hoverClosestCartesian',
        'hoverCompareCartesian',
        'zoomInGeo',
        'zoomOutGeo',
        'resetGeo',
        'hoverClosestGeo',
        'hoverClosestGl2d',
        'hoverClosestPie',
        'toggleHover',
        'resetViews',
        # 'toImage',
        'sendDataToCloud',
        'toggleSpikelines',
        'resetViewMapbox'
    ]
)